# Guider mode image reduction
Craig Lage - 02-Aug-24

In [ ]:
import os
import shlex, subprocess
import matplotlib.pyplot as plt
from astropy.io import fits
from lsst.resources import ResourcePath

In [ ]:
dayObs = 20240807
seqNum = 9
raft = 'R22'
ccd = 'S00'
filename = f"s3://rubin-summit/LSSTComCam/{dayObs}/CC_O_{dayObs}_{seqNum:06d}/CC_O_{dayObs}_{seqNum:06d}_{raft}_{ccd}_guider.fits"
rp = ResourcePath(filename)
with rp.open(mode="rb") as f:
    hdu_list = fits.open(f)

In [ ]:
hdu_list[0].header

In [ ]:
hdu_list[2].header

In [ ]:
data = hdu_list[4].data

In [ ]:
plt.imshow(data, interpolation='nearest')

In [ ]:
expId = int(f"{dayObs}{seqNum:05d}")
dirName = f"/home/c/cslage/u/ComCam/guider_mode/movie_{expId}"
%mkdir -p {dirName}
movieName = f"Guider_{expId}.mp4"
print(movieName)

# Build the individual frames
fig = plt.figure(figsize=(8, 8))
nStamps = int(hdu_list[0].header['N_STAMPS'])
for n in range(nStamps):
    data = hdu_list[2 * n + 2].data
    plt.imshow(data, interpolation='nearest')
    plt.title(f"ComCam guider mode {expId}, Frame {n+1}", fontsize=12)
    plt.savefig(f"{dirName}/Frame_{n:03d}.png")
    plt.clf()


In [ ]:
print(f"\033[1mThe movie name will be: {dirName}/{movieName}\033[0m")

command = f"ffmpeg -pattern_type glob -i '{dirName}/*.png' -f mp4 -vcodec libx264 -pix_fmt yuv420p -framerate 50 -y {dirName}/{movieName}"
args = shlex.split(command)
build_movie = subprocess.Popen(args)
build_movie.wait()